In [1]:
!pip install qdrant_client
!pip install datasets
!pip install tdqm
!pip install pymilvus
!pip install faiss
!pip install accelerate
!pip install transformers
!pip install -U bitsandbytes
!pip install datasets
!pip install rouge-score
!pip install pymorphy3
!pip install peft
!pip install flash_attn
!pip install langchain
!pip install langchain_community
!pip install pymorphy2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.7/327.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatibl

In [2]:
from tqdm import tqdm
from qdrant_client import QdrantClient
from qdrant_client.http import models
from datasets import load_dataset
from langchain_community.embeddings import HuggingFaceEmbeddings
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

In [3]:
qclient = QdrantClient(":memory:")
qclient.create_collection(
    collection_name='minecraft_wiki',
    vectors_config = models.VectorParams(
        size=1024,
        distance=models.Distance.COSINE
    ),
)

True

In [4]:
embeddings = HuggingFaceEmbeddings(
    model_name = 'intfloat/multilingual-e5-large',
    # берем эмбаддинги из модели e5, так как в ней оптимально
    # разбиваются русские слова для хранения в векторной бд
    model_kwargs={"device":"cuda"},
    )

<ipython-input-4-a97672cb8656>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
def vectorize_query(query):
    return embeddings.embed_query(query)

In [ ]:
!ls

In [ ]:
DATASET_PATH = "./dataset.txt"
cur_id = 0


f = open(DATASET_PATH,"r",encoding = "utf-8")
text_corpus = f.read().split("<SEP>")
for i in tqdm(range(len(text_corpus) if len(text_corpus) <= 20000 else 20000)):
    cnt = 0
    text = text_corpus[i]
    if len(text) > 1024:
        text = text[:1024]
    tmp = ""
    for line in text.split("\n"):
        if len(line) == 0:
            continue
        if len(line) < 40:
            cnt += 1
        tmp += line + '\n'
    if cnt >= 3:
        continue
    vector_batch = vectorize_query(tmp)
    qclient.upsert(
        collection_name='minecraft_wiki',
        points=models.Batch(
            payloads=[{
                "chunk":tmp,
            }],
            vectors=[vector_batch],
            ids=[i],
        )
    )
f.close()

In [5]:
model_name = 'openchat/openchat-3.5-1210'
# or
# model_name = 'RefalMachine/RuadaptQwen2.5-7B-Instruct-1M'

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, attn_implementation='sdpa', device_map='auto')
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

In [6]:
pipeline_kwargs={"max_new_tokens": 1024, "do_sample": True, "temperature": 0.2}
pipe = pipeline("text-generation",model=model, tokenizer=tokenizer, **pipeline_kwargs)
hugging_face_pipeline = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0
<ipython-input-6-1e55c9fe2d65>:3: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hugging_face_pipeline = HuggingFacePipeline(pipeline=pipe)


In [ ]:
def relevant_info_qdrant(query,qclient):
    relevant_points = qclient.query_points(
        collection_name='minecraft_wiki',
        query=vectorize_query(query),
        limit=3,
        with_vectors=False,
        with_payload=True
    ).points
    relevant_results = []
    for point in relevant_points:
        relevant_results.append(point.payload["chunk"])
    return relevant_results

In [ ]:
# relevant_docs
# relevant_points.[0].payload["context"][1]["chunk"]

In [7]:
def apply_chat(messages):
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt
chain = apply_chat | hugging_face_pipeline


In [43]:
questions = [
    "Как скрафтить верстак в Minecraft?",
"Какие материалы нужны для создания кирки в minecraft без модификаций, и какие из них самые прочные?",
"Как приручить волка в Minecraft?",
"Какие существуют способы добычи обсидиана без использования алмазной кирки в Minecraft без модификаций?",
"Как работает редстоун-повторитель minecraft и для чего он используется?",
"""В каких биомах можно найти редкие структуры, такие как "Древний город" или "Океанский монумент" в Minecraft?""",
"Какие мобы появляются только в определенных биомах (например, снежные лисы, коричневые панды) в Minecraft?",
"Как вызвать Рейд в деревне Minecraft, и какие условия для этого нужны?",
"Какие блоки нельзя сдвинуть поршнем в Minecraft?",
"Как работает механика разведения животных в Minecraft?",
"Как создать бесконечный источник воды в Minecraft?",
"Какие существуют способы автоматической фермы криперов Minecraft?",
"Как работает компаратор minecraft?",
"Какие блоки могут передавать редстоун-сигнал через себя в Minecraft без модификаций?",
"Как сделать движущийся объект из блоков (например самолет) в Minecraft без модификаций?",
"Как активировать портал в Энд в Minecraft, и какие предметы для этого нужны?",
"Какие стратегии можно использовать для победы над Эндер-драконом в minecraft?",
"Как работает бартер в Незере Minecraft, и для чего он нужен?",
"Какие есть способы быстро перемещаться в Незере Minecraft?",
"Как летать без читов и креатива в Minecraft?",
]

In [44]:
query = questions[15]
# results_text = '\n\n'.join(relevant_info_qdrant(query,qclient))
prompt = f"Дай ответ на вопрос. Вопрос: {query}"
# prompt = f"Дай ответ на вопрос, используя релевантную информацию.\n\nРелевантная информация:{results_text}\n\nВопрос: {query}"
chat = [
    {"role": "user", "content": prompt},
]
query

'Как активировать портал в Энд в Minecraft, и какие предметы для этого нужны?'

In [ ]:
res= chain.invoke(chat)
res[res.rfind("GPT4 Correct Assistant:"):]

'GPT4 Correct Assistant: Для активирования портала в Энд (The End) в Minecraft, вам потребуется следующее:\n\n1. Собрать 12 эндеритов (Enderman) в Энде. Это можно сделать, собрав 20 эндеритов и убив их.\n2. Собрать 12 энхантиардов (Enderman-повелители) в Энде. Это можно сделать, собрав 20 энхантиардов и убив их.\n3. Собрать 12 шары энхантиара (End Crystal) в Энде. Это можно сделать, собрав 20 шары энхантиара и убив их.\n\nПосле того, как вы собрали необходимое количество этих предметов, вы можете активировать портал в Энд:\n\n1. Подойдите к порталу и нажмите на него, чтобы открыть его инвентарь.\n2. В инвентаре портала вы увидите слоты для эндеритов, энхантиардов и шары энхантиара. В каждый слот необходимо вставить соответствующие предметы.\n3. После того, как вы вставили все предметы, портал автоматически активируется, и вы сможете переместиться в Энд.\n\nПримечание: Этот процесс может быть сложным, так как эндериты и энхантиарды могут быть опасными, поэтому рекомендуется собрать их с

Чтобы скрафтить верстак в Minecraft, вам необходимо собрать 4 блока деревянных досок и поместить их в верстак.

Для создания кирки в игре можно использовать следующие материалы:\n\n1. Железо (IRON)\n2. Золото (GOLD)\n3. Алмаз (DIAMOND)\n4. Изумруд (EMERALD)\n5. Дерево (WOOD)\n6. Камень (STONE)\n\nСамыми прочными материалами для кирки являются алмаз (DIAMOND) и изумруд (EMERALD), так как они позволяют добывать блоки наиболее эффективно и быстро.

Чтобы приручить волка в Minecraft, вам необходимо использовать кость. Найдите волка в игровом мире и нацелейте на него, щелкнув левой кнопкой мыши. Затем выберите опцию "Приручить" и бросьте кость в направлении волка. После этого волк станет вашим управляемым животным и будет следовать за вами.

Единственный способ добычи обсидиана без использования алмазной кирки - это использование незеритовой кирки. В этом случае время добычи составит 8,35 секунд. Любые другие инструменты или руки не позволяют добыть обсидиан, а только разрушают его без дропа.

Редстоун-повторитель (Redstone repeater) - это блок, используемый в игре Minecraft для увеличения времени прохождения сигнала красного камня. Он позволяет увеличить расстояние, на которое передается сигнал, а также контролировать точность времени передачи сигнала. Редстоун-повторитель используется для создания различных механических систем, таких как логические цепи, автоматические двери, механизмы транспортировки и другие конструкции.

Древние города можно найти во всех биомах, кроме биомы пустыни, а Океанские монументы можно найти только в биоме глубокого океана.

Снежные лисы и коричневые панды являются примерами мобов, которые появляются только в определенных биомах. Снежные лисы могут быть найдены в снежной тайге, а коричневые панды - в тайге, подзолистой тайге и снежной тайге.

Чтобы вызвать рейд в деревне, вам нужно выполнить следующие условия:\n\n1. Убедитесь, что на сервере установлен WorldGuard.\n2. Защитите деревню, поместив блоки вокруг нее и поставив флаг запрета на вход, если у вас есть права на это. В противном случае, обнесите деревню стеной.\n3. Убедитесь, что в деревне есть жители, иначе рейд не спаунится.\n4. На сервере должен быть установлен плагин, отвечающий за спавн мобов, например, MobArena или Towny Raid.\n5. Выполните команду или выполните действие, которое вызывает рейд, например, использование специального шарда или нападение на деревню.\n\nПосле выполнения этих условий, волны мобов (обитателей) должны спаawnить рядом с деревней и атаковать ее.

Нельзя сдвинуть поршнем следующие блоки в Minecraft:\n\n1. Абсолютно неподвижные блоки: коренная порода, наковальня, обсидиан.\n2. Хрупкие в движении блоки: тыква, кактус, кровать, дверь, расцветающий аметист и другие.\n\nКроме того, поршни не могут передвинуть блоки, прикреплённые к другим блокам, если они будут отсоединены и выпадут как предмет.

Механика разведения животных в данном случае работает следующим образом: куры несут яйца, которые по сети воронок попадают в выбрасыватель, который передаёт их в раздатчик. Затем яйца выбрасываются внутрь птицефермы и разбиваются с вероятностью 12,5%, из которых появляется цыплёнок. Численность кур растет в геометрической прогрессии, что может привести к падению производительности. Для работы размножателя необходимы две клетки, в каждой из которых находится отдельное животное. Клетки соединяются вместе, и на месте стыка происходит кормление животных, после чего у них появляется детеныш, который транспортируется в выбранное место посредством воды.

Чтобы создать бесконечный источник воды в Minecraft, нужно разместить блоки воды в формате 3×1, 2×2 и более, чтобы вода сливалась из одного блока в другой, создавая непрерывный поток.

Существуют два основных способа автоматической фермы криперов:\n\n1. Сбор древесины с помощью криперов:\n\nДля автоматической добычи древесины с помощью криперов, вам потребуется создать систему, которая будет обеспечивать постоянный поток криперов на вашей ферме. Это может быть достигнуто с помощью автоматических транспортных систем, таких как пояс, портал или конвейеры. Кроме того, вам потребуется обеспечить безопасность от взрывов, используя блоки, такие как столбы или стены из обычного камня вокруг фермы.\n\n2. Сбор древесины с помощью гастов:\n\nВторой способ - использовать гастов для автоматической добычи древесины. Для этого вам потребуется создать ферму с гастами и обеспечить ее автоматическую работу с помощью автоматических транспортных систем, таких как пояс, портал или конвейеры. Гасты могут быть призваны с помощью яиц призывания, и их можно использовать для автоматической добычи древесины без риска взрывов.\n\nОба этих способа позволяют автоматически собирать древесину на ферме, но каждый имеет свои преимущества и недостатки. Важно учитывать эти факторы при выборе подходящего метода для вашей фермы

Компаратор - это специальный блок в схемах из красного камня, который позволяет сравнивать два сигнала красного камня между собой, вычитать один сигнал из другого, проверять заполненность контейнеров и выдавать сигнал в зависимости от параметров некоторых блоков, расположенных позади него.

Блоки, которые могут передавать редстоун-сигнал через себя, включают:\n\n1. Редстоун-трубу (Redstone Torch) - используется для передачи сигнала между блоками.\n2. Редстоун-лампа (Redstone Lamp) - используется для регулирования сигнала и его распространения.\n3. Реле (Sticky Piston) - используется для передачи сигнала и активации механизмов.\n4. Редстоун-блок (Redstone Block) - источник постоянного сигнала, используемый для построения схем и активации механизмов.\n5. Фрейм с редстоун-трубой (Redstone Repeater) - специальный блок, необходимый для создания передвижных конструкций, используемых для передачи сигнала.\n\nЭти блоки позволяют создавать сложные и интересные механизмы и схемы в игре Minecraft

Для создания движущегося объекта из блоков, такого как самолет, вам нужно использовать механизм анимации в программном коде. В большинстве случаев, это будет выполняться с помощью фреймворка, такого как Unity или Unreal Engine.\n\nВам нужно будет создать 3D-модель самолета и применить анимацию к нему. Затем, используя скрипты, вы можете контролировать движение самолета по заданной траектории или в заданном направлении.\n\nВажно учитывать, что для создания анимации и движения объекта из блоков, вам нужно иметь базовые знания в программировании и использовании фреймворков для создания игр.

Для активирования портала в Энд необходимо использовать материал - око Эндера. Для этого следуйте этим шагам:\n\n1. Соберите око Эндера, используя карту Эндера и око Края.\n2. Постройте портал в льдинный мир, как обычный портал в Нижний мир.\n3. Нажмите на нижний блок портала с помощью скипетра.\n4. Вылить ведро лавы или вывести поток лавы к доске так, чтобы лава ее касалась.\n5. Через некоторое время доска загорится и активирует портал в Энд.\n\nТаким образом, для активации портала в Энд нужны: око Эндера, ведро лавы и скипетр

ля победы над Эндер-драконом можно использовать следующие стратегии:\n\n1. Использовать снежки для уничтожения кристаллов Края во время боя.\n2. Спрятаться в ведре рыхлого снега или использовать элитры, чтобы укрыться от атак дракона.\n3. Выпить зелье медленного падения, чтобы снизить скорость падения и избежать смертельного удара при падении с большой высоты.\n4. Кинуть эндер-жемчуг на ближайшую твердую поверхность перед смертью, чтобы собрать вещи после респауна.\n5. Атаковать дракона, когда он летает вдали, используя стрелы из лука и переключаясь на меч, когда он подлетает близко.\n6. Оставаться в безопасности от кислоты Края, избегая зон, где дракон поливает поле окружающей среды.\n7. Замечать, что на дракона не действуют эффекты и чары, поэтому сосредоточьтесь на физических атаках и уклонении от его ударов.

Бартер в Незере - это механика, которая позволяет игроку обмениваться предметами на золотые слитки и "кота в мешке" с пиглинами. Для этого игрок может использовать ресурсы и предметы, которые он находит или создает, на пиглиновых торговцев. Затем игрок может обмениваться этими золотыми слитками на незеритовые слитки, которые имеют различные применения в игре.\n\nНезеритовый слиток может использоваться для улучшения алмазных инструментов и брони, а также для использования как топлива для маяка. Таким образом, бартер в Незере является важным инструментом для игроков, которые хотят улучшить свои предметы и получить доступ к другим полезным вещам в игре.

Для быстрого перемещения в Незере можно использовать транспортную систему Нижнего мира, которая организована на основе релевантной информации. Одним из способов быстрого перемещения является использование обсидиана для организации быстрой смены направления движения и перевозки вещей между базами.

Для полёта в Minecraft без использования читов и в режиме креатив необходимо построить летательный аппарат, используя блоки и механизмы, доступные в игре. Один из простых способов - это создать летающий блок с помощью мода Spigot или Bukkit. Другой вариант - это использовать блок ангела из мода Extra Utilities, чтобы создать летающие сооружения.

БЕЗ РАГ

Для скрафтинга верстака в Minecraft, вам потребуется следующее:\n\n1. Медный блок: вы можете получить его, разрушив медную руду с помощью кирки или других блоков. Медная руда встречается в пещерах, поверхностных шахтах и в джунглях.\n2. Деревянная лопата: вы можете получить ее, разрушив дерево с помощью кирки или других блоков.\n3. Деревянный пик: вы можете получить его, разрушив дерево с помощью кирки или других блоков.\n\nСоберите все эти предметы и откройте меню ремесленника (Crafting Table). Расположите медный блок в центре, а деревянную лопату и деревянный пик вокруг него, образуя квадрат. После этого вы сможете получить верстак.

Для создания кирки в Minecraft без модификаций, вам понадобятся следующие материалы:\n\n1. Дерево: Для создания ручки кирки. Любой тип дерева будет работать, но самый прочный из них - акация.\n\n2. Кованая сталь: Для создания лезвия кирки. Вам нужно собрать печь для ковки и использовать ее для переработки стали в кованую сталь.\n\nСамые прочные материалы для создания кирки в Minecraft без модификаций - это акациевая ручка и кованная сталь

В Minecraft нет возможности приручить волка. Волки являются дикими животными, и их можно только ловить с помощью лыка или удерживать их, когда они атакуют игрока. Однако, вы можете построить печку или огонь рядом с волком, чтобы он приблизился к вам и стал дружелюбным. Однако, волки все еще будут агрессивно реагировать на других игроков, поэтому нельзя считать, что вы приручили волка.

В Minecraft без модификаций, для добычи обсидиана без использования алмазной кирки, можно воспользоваться следующими способами:\n\n1. Использование пики: Пики могут использоваться для добычи обсидиана, но они будут быстро изнашиваться и потребоваться часто заменять.\n\n2. Использование дробилки: Дробилка может использоваться для добычи обсидиана, но она будет работать медленно и потребует много времени.\n\n3. Использование других инструментов: Также можно попробовать использовать другие инструменты, такие как лопата или папайя, но они не такие эффективны, как алмазная кирка.\n\nОднако, использование алмазной кирки является наиболее эффективным и быстрым способом добычи обсидиана в Minecraft без модификаций.

Редстоун-повторитель (Redstone Repeater) в игре Minecraft является блоком, который используется для управления сигналами редстоуна. Он повторяет сигнал на определенном интервале времени, который можно настроить с помощью приложения (Attachment) на блоке. Редстоун-повторитель используется для создания различных механизмов и устройств, таких как логические цепи, автоматические двери, системы предупреждения и многие другие. Он позволяет создавать задержку или периодическое повторение сигналов, что делает его важным инструментом для проектирования сложных механизмов в Minecraft.

В Minecraft редкие структуры, такие как "Древний город" и "Океанский монумент", можно найти в биомах, где есть песок и гравий, таких как Песочные дюны, Саванна и Плато.

В Minecraft некоторые мобы появляются только в определенных биомах. Например:\n\n1. Снежные лисы (Snowy Taigas) - появляются в тайгах с снегом.\n2. Коричневые панды (Bamboo Jungle) - появляются в бамбуковых джунглях.\n3. Деревенские коты (Savanna) - появляются в саваннах.\n4. Летучие змеи (Mesa) - появляются в меса биомах.\n5. Акулы (Ocean) - появляются в океанах.\n6. Животные, обитающие в пещерах (Caves) - такие как зомби-пауки, дроуны и другие.\n\nЭто не исчерпывающий список, но эти примеры иллюстрируют, как определенные мобы связаны с определенными биомами в Minecraft.

Для вызова Рейда в деревне Minecraft, вам нужно выполнить следующие условия:\n\n1. Уровень опыта: Ваш персонаж должен иметь минимум 30 уровня опыта.\n2. Строительство: Вам нужно построить деревню из минимум 10 домов, каждый из которых должен иметь минимум 3 блока в высоту.\n3. Население: В деревне должно быть минимум 5 жителей (не обязательно реальных игроков). Жители могут быть созданы с помощью команд или появиться случайно, если другие игроки посетят вашу деревню.\n4. Условия для Рейда: Деревня должна быть расположена в диком биоме, таком как лес, саванна или пустыня.\n\nКогда все условия выполнены, Рейд автоматически вызовется, когда ваш персонаж будет рядом с деревней. Рейды могут состоять из разных типов монстров, таких как зомби, скелеты, дроуны и другие, в зависимости от биома, в котором расположена деревня.

В Minecraft нельзя сдвинуть поршнем такие блоки, как:\n\n1. Ограждение (Fence)\n2. Дверь (Door)\n3. Торшер (Trapdoor)\n4. Лягушка (Vine)\n5. Лиана (Climbing_vine)\n6. Дерево (Tree)\n7. Листья дерева (Leaves)\n8. Мхи (Moss_block)\n9. Железобетон (Concrete)\n10. Кирпич (Bricks)\n11. Стена из кирпича (Brick_wall)\n12. Глиняный горн (Clay_furnace)\n13. Мебель (Furniture) - столы, шкафы, книжные полки и т.д.\n14. Подставка для свитков (Loom)\n15. Компас (Compass)\n16. Барабаны (Drums)\n17. Механизм (Mechanism)\n18. Генератор (Generator)\n19. Компас (Compass)\n20. Железный поршень (Iron_plate)\n\nЭтот список не является исчерпывающим, но включает в себя основные блоки, которые нельзя сдвинуть поршнем.

В Minecraft, механика разведения животных работает следующим образом:\n\n1. Чтобы разводить животных, вам нужно иметь еду, которую они могут поесть. Например, для овец вам нужно иметь во Wolves sheep wool, для свиней - картофель или яблоки, а для кроликов - еду, которую можно получить, разрушив снежные блоки.\n\n2. Найдите животное, которое вы хотите разводить. Вы можете найти их в дикой природе или создать их в специальных блоках, таких как "Кресло с патронём" для коров, свиней и овец, или "Кресло с патронём (снежный)" для кроликов.\n\n3. Подойдите к животному и нажмите на него, чтобы открыть меню инвентаря. Затем выберите еду, которую вы хотите отдать животному, и перетащите её в специальную ячейку.\n\n4. Животное будет есть, и оно начнёт любить вас. Чем больше еды вы дадите животному, тем больше оно любит вас. Когда у животного будет максимальное уровень любви (10 уровней), оно станет доступным для разведения.\n\n5. Чтобы разводить животное, просто щелкните на нем, чтобы открыть меню разведения. Затем выберите животное, с которым вы хотите спарить его, и нажмите на кнопку "Спарить".\n\n6. После спаривания, вам нужно подождать, пока не появится детёныш. Время, которое требуется на размножение, зависит от вида животного.\n\n7. После рождения детёныша, вы можете продолжать разводить животных и получать ещё больше детёнышей.\n\nОбратите внимание, что некоторые виды животных могут быть защищены от разведения, например, если они имеют неправильную версию игры или если они были созданы с помощью модов.

Для создания бесконечного источника воды в Minecraft, вы можете использовать следующие шаги:\n\n1. Создайте портал в Огненной пещере. Для этого нужно найти Огненный коридор, который соединяет Огненную пещеру с другой пещерой, и построить портал из 8 блоков Огня в форме квадрата.\n\n2. Подойдите к порталу и нажмите на него, чтобы пройти внутрь. Вы появитесь в другой пещере, где найдете большое количество воды.\n\n3. Создайте водоем из блоков Воды внутри пещеры. Вы можете использовать буквы "В" для быстрого создания водоема.\n\n4. Поместите в водоем блок "Синяя дыра" (#) в центре. Это позволит создать бесконечный источник воды.\n\n5. Теперь вы можете собирать воду из этого бесконечного источника, используя букву "В" или рычаг "Перекачиватель воды".\n\nОбратите внимание, что бесконечный источник воды будет работать только в том случае, если вы находитесь внутри пещеры. Если вы покинете пещеру, источник воды перестанет работать.

В Minecraft существует несколько способов создания автоматической фермы криперов. Один из наиболее популярных и простых в реализации способов - это использование системы с транспортировкой, которая включает в себя следующие компоненты:\n\n1. Система транспортировки: Для создания транспортной системы можно использовать ленты транспортировки из мода "MovingPistons" или "PistonTransport". Эти моды позволяют создавать ленты, которые перемещают криперов от одного места на другое, используя пистоны.\n\n2. Зоны размножения: В зонах размножения криперы собираются в группы и размножаются. Для этого можно использовать обычные методы размножения криперов, такие как использование криперов с разными цветами или разных видов.\n\n3. Система сбора: После того, как криперы размножаются, их нужно собрать для использования в качестве топлива или продажи. Для этого можно использовать автоматические системы сбора, которые собирают криперы и помещают их в инвентарь игрока или в специальный контейнер.\n\n4. Система питания: Для обеспечения непрерывного размножения криперов, нужно обеспечить постоянный поток пищи. Для этого можно использовать автоматические системы производства пищи, такие как автоматические фермы картофеля или рыбы.\n\nЭтот способ автоматической фермы криперов является одним из наиболее простых и эффективных способов в Minecraft. Однако, существуют и другие способы, которые могут быть более сложными, но более эффективными в определенных ситуациях.

Компаратор в Minecraft - это инструмент, который позволяет объединять идентичные предметы в одну копию. Когда вы используете компаратор, он сравнивает предметы в инвентаре игрока и объединяет идентичные предметы в одну копию. Это полезно, когда игрок хочет убрать лишние предметы или собрать большое количество одинаковых предметов в одну копию.\n\nЧтобы использовать компаратор, вам нужно иметь его в инвентаре. Вы можете получить компаратор, найдя его в chest, который находится в деревне заточения в Minecraft. Когда вы имеете компаратор, вы можете его использовать, нажав на него, и затем выбрав два или более идентичных предмета в инвентаре, которые вы хотите объединить. Компаратор автоматически объединит эти предметы в одну копию.\n\nОбратите внимание, что компаратор не может объединять идентичные предметы, которые находятся в контейнерах, таких как сумки или барлыки. Для объединения таких предметов вам нужно использовать другой инструмент, называемый переносчик.

В Minecraft есть несколько блоков, которые могут передавать редстоун-сигнал через себся без модификаций. Эти блоки включают:\n\n1. Обычный редстоун\n2. Полоса редстоуна\n3. Редстоун-торч\n4. Редстоун-лампа\n5. Редстоун-репетер\n6. Редстоун-эффектор\n\nОбратите внимание, что некоторые из этих блоков могут изменять сигнал, например, редстоун-лампа преобразует сигнал в свет, но они все же могут передавать редстоун-сигнал через себя.

Для создания движущегося объекта из блоков, например самолета, в Minecraft без модификаций, вам понадобятся следующие шаги:\n\n1. Создайте основную структуру самолета из блоков. Используйте блоки, которые напоминают самолет, такие как крылья, фюзеляж и хвост.\n\n2. Создайте движущуюся часть самолета. Для этого вам понадобятся механизмы, которые будут двигать самолет. Вот несколько идей:\n\na. Используйте пистолет с стрельбой вперед/назад, чтобы двигать самолет вперед и назад. Для этого вы должны построить пистолет и прикрепить его к самолету таким образом, чтобы его выстрелы двигали самолет.\n\nb. Используйте таймеры и пистолеты с стрельбой вперед/назад, чтобы создать цикл движения самолета вперед и назад или вверх и вниз. Настройте таймеры таким образом, чтобы пистолеты стреляли по очереди, создавая движение.\n\nc. Используйте механические системы, такие как системы с переключателями, чтобы контролировать движение самолета. Например, вы можете создать систему, которая переключает между разными пистолетами с стрельбой вперед/назад, чтобы создать более сложные движения.\n\n3. Тестируйте и настраивайте движение самолета. После того, как вы построили механизмы, запустите самолет и настройте их, чтобы добиться желаемого движения.\n\n4. Добавьте эффекты и детализацию. Чтобы сделать самолет более реалистичным, добавьте эффекты, такие как дым или взмахи крыльев, и детализацию, такую как движущиеся винты или движущиеся части самолета.\n\nПомните, что создание движущегося объекта из блоков в Minecraft без модификаций может потребовать много времени и творчества, но с использованием этих шагов вы сможете создать удивительный самолет, который будет двигаться по вашему усмотрению.


Как скрафтить верстак в Minecraft?
Какие материалы нужны для создания кирки в minecraft без модификаций, и какие из них самые прочные?
Как приручить волка в Minecraft?
Какие существуют способы добычи обсидиана без использования алмазной кирки в Minecraft без модификаций?
Как работает редстоун-повторитель minecraft и для чего он используется?
В каких биомах можно найти редкие структуры, такие как "Древний город" или "Океанский монумент" в Minecraft?
Какие мобы появляются только в определенных биомах (например, снежные лисы, коричневые панды) в Minecraft?
Как вызвать Рейд в деревне Minecraft, и какие условия для этого нужны?
Какие блоки нельзя сдвинуть поршнем в Minecraft?
Как работает механика разведения животных в Minecraft?
Как создать бесконечный источник воды в Minecraft?
Какие существуют способы автоматической фермы криперов Minecraft?
Как работает компаратор minecraft?
Какие блоки могут проводить редстоун-сигнал в Minecraft без модификаций?
Как сделать движущийся объект из блоков (например самолет) в Minecraft без модификаций?
Как активировать портал в Энд в Minecraft, и какие предметы для этого нужны?
Какие стратегии можно использовать для победы над Эндер-драконом в minecraft?
Как работает бартер в Незере Minecraft, и для чего он нужен?
Какие есть способы быстро перемещаться в Незере Minecraft?
Как летать без читов и креатива в Minecraft?

In [ ]:
query = "Как скрафтить верстак в Minecraft?"
# Это проверка базы данных
# query = dataset["question"][7]
print(query)
relevant_points = qclient.query_points(
        collection_name='minecraft_wiki',
        query=vectorize_query(query),
        limit=3,
        with_vectors=False,
        with_payload=True
    ).points
relevant_points

In [ ]:
#Тут будет 20 вопросов И их задание в ллм